In [1]:
from time import sleep

from IPython.display import clear_output

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import json

import pandas as pd

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(chrome_options=options)

In [3]:
cities = pd.read_csv('../Population/rosstat_clean.csv', encoding='cp1251')

In [ ]:
cities_info = []
for ind, city in cities.iterrows():
    # Для некороых городов надо вручную менять часть для ссылки, так как иначе
    # откроется неправильная страница в википедии
    if(city['City'] == 'Владимир'): urlpart = 'Владимир_(город,_Россия)'
    elif (city['City'] == 'Орел'): urlpart = 'Орёл_(город)'
    elif (city['City'] == 'Курган'): urlpart = 'Курган_(город)'
    else: urlpart = city['City']
    print(city['City'])
    city_info = {}
    browser.get('https://ru.wikipedia.org/wiki/' + urlpart)
    trlist = browser.find_elements(By.TAG_NAME,'tr')
    country = ''
    region = ''
    for i in range(0, len(trlist)):
        parts = trlist[i].text.split(' ')
        match parts[0]:
            case 'Страна': 
                country = (lambda p: p[2] if p[1] == '' else p[1])(parts) if (country == '') else country
            case 'Субъект': 
                for part in parts: 
                    if (part != 'Субъект' and part != 'Федерации'):
                        region += part + ' '
            # Для Севастополя и Симферополя структура общей информации о городе отличается от других городов.
            # Приходится обрабатывать отдельно
            case 'Регион':
                for part in parts: 
                    if (part != 'Регион'):
                        region += part + ' '
                region = region.split('[b]')[0] 
    region = region[:-1] if (region[-1] == ' ') else region
    country = country[:-1] if (country[-1] == '/') else country
    city_info = {'name': city['City'], 'country': country, 'region': region}
    cities_info.append(city_info)
    sleep(5)

In [5]:
with open('address.json', 'w') as file:
    json.dump(cities_info,file, indent = 3, ensure_ascii = False)